<a href="https://colab.research.google.com/github/SkeTJ/SteamGameRecommender/blob/main/SteamGameRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import difflib

# Load the Drive helper and mount
from google.colab import drive
import os.path
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
api_key = ''
steam_key = {'key': api_key, 'max_results': 50000}
def get_app_list():
  # Get initial batch of list
  store_app_list_url = 'https://api.steampowered.com/IStoreService/GetAppList/v1/'
  app_list = requests.get(url = store_app_list_url, params = steam_key).json()

  df = pd.DataFrame(app_list)
  df = json_normalize(df['response']['apps'])

  # Get Last App ID to complete the DataFrame
  last_appid = df['appid'].iloc[-1]
  df_list = [df]
  while True:
    appid_params = {'key': api_key, 'last_appid': last_appid,'max_results': 50000}
    new_df = requests.get(url = store_app_list_url, params = appid_params).json()
    new_df = pd.DataFrame(new_df)
    if new_df.empty:
      break

    new_df = json_normalize(new_df['response']['apps'])

    # Append the new data to a DataFrame
    df_list.append(new_df)

    # Iterate to the next last App ID
    last_appid = new_df['appid'].iloc[-1]

  df = pd.concat(df_list)

  # Remove Unnecessary Columns
  df = df.drop(['last_modified', 'price_change_number'], axis = 1)
  df = df.dropna()

  return df

In [4]:
appid_df = get_app_list()

In [5]:
appid_df

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force
...,...,...
40889,2615670,Bewitching Sinners
40890,2616050,Dual Bus Simulator
40891,2616060,Blackhole on the Road
40892,2616080,BUBG Single on the Ground


In [6]:
# LOWER THE DATASET FOR NOW, TO BE REMOVED
halved_df = appid_df[:-90500]

In [7]:
halved_df.head(5)

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


In [8]:
halved_df.tail(5)

,appid,name
389,15930,Luxor 3
390,15960,Little Farm
391,15970,Luxor
392,16000,Discovery! A Seek and Find Adventure
393,16020,Samantha Swift and the Hidden Roses of Athena


# Get the specific details of each app based on the App ID

In [9]:
# Requests will be made in two since the app details is in one API
# requests whilst the review is also in a different API request.
def get_app_details(appid_df):
  # Prepare CSV file
  csv_file_name = "steam_app_list.csv"
  csv_file_location = f''
  remove_header = False

  # Check whether CSV file exists
  if os.path.exists(csv_file_location):
    steam_app_df = pd.read_csv(csv_file_location)
    last_appid = steam_app_df['steam_appid'].iloc[-1]

    # Since the CSV exist, we do not need to apply the header again
    remove_header = True

    # Get the last index of the CSV
    temp_df = appid_df.loc[appid_df['appid'] == last_appid]
    last_index = temp_df.index[0]

    # Get the last App ID index and remove irrelevant rows
    appid_df = appid_df[last_index + 1:]
    appids = appid_df.appid
  else:
    appids = appid_df.appid

  game_list = []
  for appid in appids:
    app_detail_url = f'http://store.steampowered.com/api/appdetails?appids={appid}'
    app_review_url = f'https://store.steampowered.com/appreviews/{appid}?json=1&language=all&purchase_type=all&num_per_page=0'

    app_detail = requests.get(url = app_detail_url).json()
    app_review = requests.get(url = app_review_url).json()['query_summary']
    try:
      app_df = pd.DataFrame(app_detail)
      app_df = json_normalize(app_df[f'{appid}'])
      review_df = pd.json_normalize(app_review)

      # Clean Review DataFrame before attaching to main App DataFrame
      review_df = review_df.drop(['num_reviews'], axis = 1)

      # WORKAROUND
      # The DataFrame always return a 2nd row (1 in the index) with all NaN values
      # So I figured that it should just be removed everytime the it is made into a DataFrame
      app_df = app_df.drop(1)

      # Append both App and Review DataFrame together
      app_df = pd.concat([app_df, review_df], axis = 1)

      game_list.append(app_df)
    except:
      continue

  df = pd.concat(game_list)
  df = df.reset_index()
  df = df.drop(['index'], axis = 1)

  # Append to the CSV
  if remove_header == True:
    # Resume from the last index
    df.index = np.arange(last_index, last_index  + (len(df)))
    df.to_csv(csv_file_location, mode = 'a', header = None)
  else:
    df.to_csv(csv_file_location, mode = 'a')

  return df

In [10]:
apps_df = get_app_details(halved_df)

In [11]:
apps_df

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,achievements.highlighted,pc_requirements.recommended,controller_support,reviews,linux_requirements.recommended,legal_notice,pc_requirements,drm_notice,mac_requirements.recommended,ext_user_account_notice
0,game,Counter-Strike,10.0,0.0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,Team Fortress Classic,20.0,0.0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,game,Day of Defeat,30.0,0.0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,game,Deathmatch Classic,40.0,0.0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,game,Half-Life: Opposing Force,50.0,0.0,False,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,game,Luxor 3,15930.0,0.0,False,"A brave young Pharaoh, killed prematurely by a...","A brave young Pharaoh, killed prematurely by a...","A brave young Pharaoh, killed prematurely by a...",English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,game,Little Farm,15960.0,0.0,False,"Have a down-home good time at Little Farm, whe...","Have a down-home good time at Little Farm, whe...","Have a down-home good time at Little Farm, whe...",English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,game,Luxor,15970.0,0.0,False,"Addictive and exciting, Luxor is an action-puz...","Addictive and exciting, Luxor is an action-puz...","Addictive and exciting, Luxor is an action-puz...","English, French, German, Swedish",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,game,Discovery! A Seek and Find Adventure,16000.0,0.0,False,Break out your magnifying glass and passport-I...,Break out your magnifying glass and passport-I...,Break out your magnifying glass and passport-I...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
apps_df['recommendations.total'][0]

139169.0

In [13]:
apps_df['detailed_description'].head(5)

0    Play the world's number 1 online action game. ...
1    One of the most popular online action games of...
2    Enlist in an intense brand of Axis vs. Allied ...
3    Enjoy fast-paced multiplayer gaming with Death...
4    Return to the Black Mesa Research Facility as ...
Name: detailed_description, dtype: object

In [14]:
apps_df['developers'].head(5)

0               [Valve]
1               [Valve]
2               [Valve]
3               [Valve]
4    [Gearbox Software]
Name: developers, dtype: object

In [15]:
apps_df['categories'].head(5)

0    [{'id': 1, 'description': 'Multi-player'}, {'i...
1    [{'id': 1, 'description': 'Multi-player'}, {'i...
2    [{'id': 1, 'description': 'Multi-player'}, {'i...
3    [{'id': 1, 'description': 'Multi-player'}, {'i...
4    [{'id': 2, 'description': 'Single-player'}, {'...
Name: categories, dtype: object

In [16]:
apps_df['genres'].head(5)

0    [{'id': '1', 'description': 'Action'}]
1    [{'id': '1', 'description': 'Action'}]
2    [{'id': '1', 'description': 'Action'}]
3    [{'id': '1', 'description': 'Action'}]
4    [{'id': '1', 'description': 'Action'}]
Name: genres, dtype: object

In [17]:
# Unpack categories
def get_list(x):
  if isinstance(x, list):
    descriptions = [i['description'] for i in x]
    if len(descriptions) > 5:
      descriptions = descriptions[:5]
    return descriptions

  # If there are no description provided, just return nothing
  return []

In [18]:
features = ['categories', 'genres']
for feature in features:
  apps_df[feature] = apps_df[feature].apply(get_list)

In [19]:
apps_df[['name', 'developers', 'categories', 'genres']].head(5)

,name,developers,categories,genres
0,Counter-Strike,[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action]
1,Team Fortress Classic,[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action]
2,Day of Defeat,[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action]
3,Deathmatch Classic,[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action]
4,Half-Life: Opposing Force,[Gearbox Software],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action]


In [20]:
def clean_data(x):
  if isinstance(x, list):
    return [str.lower(i.replace(" ", "")) for i in x]
  else:
    # Check if Developers exists
    if isinstance(x, str):
      return str.lower(x.replace(" ", ""))
    else:
      return ''

In [21]:
features = ['developers', 'categories', 'genres']
for feature in features:
  apps_df[feature] = apps_df[feature].apply(clean_data)

In [22]:
def create_soup(x):
  return ' '.join(x['categories']) + ' ' + ' '.join(x['developers']) + ' '.join(x['genres'])

In [23]:
apps_df['soup'] = apps_df.apply(create_soup, axis = 1)

In [24]:
apps_df['soup'].head(5)

0    multi-player pvp onlinepvp shared/splitscreenp...
1    multi-player pvp onlinepvp shared/splitscreenp...
2      multi-player valveanti-cheatenabled valveaction
3    multi-player pvp onlinepvp shared/splitscreenp...
4    single-player multi-player valveanti-cheatenab...
Name: soup, dtype: object

# Mix both Plot Description and Metadata Soup Together



In [25]:
def fit_data(apps_df):
  # Plot Description Recommender
  tfidf = TfidfVectorizer(stop_words = 'english')
  apps_df['detailed_description'] = apps_df['detailed_description'].fillna('')
  tfidf_matrix = tfidf.fit_transform(apps_df['detailed_description'])
  # print(tfidf_matrix.shape)

  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

  # Metadata Recommender
  count = CountVectorizer(stop_words = 'english')
  count_matrix = count.fit_transform(apps_df['soup'])
  # print(count_matrix.shape)

  cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
  indices = pd.Series(apps_df.index, index = apps_df['name']).drop_duplicates()

  return cosine_sim, cosine_sim2, indices

In [26]:
cosine_sim, cosine_sim2, indices = fit_data(apps_df)

In [30]:
# For the name, use difflib.get_close_matches
def get_recommendations(name, cosine_sim=cosine_sim, cosine_sim2=cosine_sim2):
  # Get similar name out of what the user type so when there is a typo, it's still fine
  similar_name = difflib.get_close_matches(word = name, possibilities = apps_df['name'], n = 10, cutoff = 0.35)

  # Unusable, spouts out error, will fix in the future
  # game_list_plot = []
  # for i in similar_name:
  #   plot_idx = indices[i]

  #   # Plot Description Recommender
  #   sim_scores_plot = list(enumerate(cosine_sim[plot_idx]))
  #   sim_scores_plot = sorted(sim_scores_plot, key = lambda x: x[1], reverse = True)
  #   sim_scores_plot = sim_scores_plot[1:11]
  #   game_indices_plot = [i[0] for i in sim_scores_plot]

  #   game_list_plot.append(apps_df['name'].iloc[game_indices_plot])

  game_list_metadata = []
  for i in similar_name:
    meta_idx = indices[i]

    # Metadata Recommender
    sim_scores_meta = list(enumerate(cosine_sim2[meta_idx]))
    sim_scores_meta = sorted(sim_scores_meta, key = lambda x: x[1], reverse = True)
    sim_scores_meta = sim_scores_meta[1:11]
    game_indices_meta = [i[0] for i in sim_scores_meta]

    game_list_metadata.append(apps_df['name'].iloc[game_indices_meta])

  # print(game_list_plot)
  # print("**************")
  # print(game_list_metadata)
  # game_list = game_list_plot.append(game_list_metadata)

  df = pd.concat(game_list_metadata)
  df = df.reset_index()
  df = df.drop(['index'], axis = 1)
  df = df.drop_duplicates()

  return df

In [31]:
rec_df = get_recommendations('Half-Life')

In [32]:
rec_df

,name
0,Left 4 Dead 2
1,Multiwinia
2,Two Worlds Epic Edition
3,Quake III Arena
4,"Star Wars: Battlefront 2 (Classic, 2005)"
5,Two Worlds II HD
6,Heretic: Shadow of the Serpent Riders
7,"Warhammer® 40,000: Dawn of War® - Dark Crusade"
8,Return to Castle Wolfenstein
9,"Warhammer® 40,000: Dawn of War® - Soulstorm"


In [40]:
def calc_baseline(apps_df):
  metacritic_mean = apps_df['metacritic.score'].mean()
  recc_quantile = apps_df['recommendations.total'].quantile(0.9)

  qualified_df = apps_df.copy().loc[apps_df['recommendations.total'] >= recc_quantile]

  return qualified_df, metacritic_mean, recc_quantile

In [41]:
qualified_df, m, C = calc_baseline(apps_df)
qualified_df

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,pc_requirements.recommended,controller_support,reviews,linux_requirements.recommended,legal_notice,pc_requirements,drm_notice,mac_requirements.recommended,ext_user_account_notice,soup
0,game,Counter-Strike,10.0,0.0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi-player pvp onlinepvp shared/splitscreenp...
4,game,Half-Life: Opposing Force,50.0,0.0,False,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player multi-player valveanti-cheatenab...
6,game,Half-Life,70.0,0.0,False,Named Game of the Year by over 50 publications...,Named Game of the Year by over 50 publications...,Named Game of the Year by over 50 publications...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player multi-player pvp onlinepvp steam...
7,game,Counter-Strike: Condition Zero,80.0,0.0,False,"With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...","English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player multi-player valveanti-cheatenab...
9,game,Half-Life 2,220.0,0.0,False,1998. HALF-LIFE sends a shock through the game...,1998. HALF-LIFE sends a shock through the game...,1998. HALF-LIFE sends a shock through the game...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements steamtradingca...
10,game,Counter-Strike: Source,240.0,0.0,False,THE NEXT INSTALLMENT OF THE WORLD'S # 1 ONLINE...,THE NEXT INSTALLMENT OF THE WORLD'S # 1 ONLINE...,Counter-Strike: Source blends Counter-Strike's...,"English, French, German, Italian, Japanese, Ko...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi-player cross-platformmultiplayer steamac...
16,game,Half-Life 2: Episode One,380.0,0.0,False,Half-Life 2 has sold over 4 million copies wor...,Half-Life 2 has sold over 4 million copies wor...,Half-Life 2 has sold over 4 million copies wor...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements captionsavaila...
17,game,Portal,400.0,0.0,False,<p>Portal&trade; is a new single player game f...,<p>Portal&trade; is a new single player game f...,Portal&trade; is a new single player game from...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements captionsavaila...
18,game,Half-Life 2: Episode Two,420.0,0.0,False,<p>Half-Life&reg; 2: Episode Two is the second...,<p>Half-Life&reg; 2: Episode Two is the second...,Half-Life&reg; 2: Episode Two is the second in...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements captionsavaila...
19,game,Team Fortress 2,440.0,0.0,True,"<p><strong>""The most fun you can have online""<...","<p><strong>""The most fun you can have online""<...",Nine distinct classes provide a broad range of...,"English<strong>*</strong>, Danish, D

In [42]:
# Function that computes the weighted rating of each game
def weighted_rating(x, m=m, C=C):
  v = x['recommendations.total']
  R = x['metacritic.score']

  # Calculation based on the IMDB formula
  return (v / (v + m) * R) + (m / (m + v) * C)

In [43]:
qualified_df['recc_score'] = qualified_df.apply(weighted_rating, axis = 1)

In [44]:
qualified_df

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,controller_support,reviews,linux_requirements.recommended,legal_notice,pc_requirements,drm_notice,mac_requirements.recommended,ext_user_account_notice,soup,recc_score
0,game,Counter-Strike,10.0,0.0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi-player pvp onlinepvp shared/splitscreenp...,95.430529
4,game,Half-Life: Opposing Force,50.0,0.0,False,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player multi-player valveanti-cheatenab...,NaN
6,game,Half-Life,70.0,0.0,False,Named Game of the Year by over 50 publications...,Named Game of the Year by over 50 publications...,Named Game of the Year by over 50 publications...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player multi-player pvp onlinepvp steam...,109.421718
7,game,Counter-Strike: Condition Zero,80.0,0.0,False,"With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...","English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player multi-player valveanti-cheatenab...,125.161259
9,game,Half-Life 2,220.0,0.0,False,1998. HALF-LIFE sends a shock through the game...,1998. HALF-LIFE sends a shock through the game...,1998. HALF-LIFE sends a shock through the game...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements steamtradingca...,103.683950
10,game,Counter-Strike: Source,240.0,0.0,False,THE NEXT INSTALLMENT OF THE WORLD'S # 1 ONLINE...,THE NEXT INSTALLMENT OF THE WORLD'S # 1 ONLINE...,Counter-Strike: Source blends Counter-Strike's...,"English, French, German, Italian, Japanese, Ko...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi-player cross-platformmultiplayer steamac...,97.649007
16,game,Half-Life 2: Episode One,380.0,0.0,False,Half-Life 2 has sold over 4 million copies wor...,Half-Life 2 has sold over 4 million copies wor...,Half-Life 2 has sold over 4 million copies wor...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements captionsavaila...,135.178111
17,game,Portal,400.0,0.0,False,<p>Portal&trade; is a new single player game f...,<p>Portal&trade; is a new single player game f...,Portal&trade; is a new single player game from...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements captionsavaila...,98.653733
18,game,Half-Life 2: Episode Two,420.0,0.0,False,<p>Half-Life&reg; 2: Episode Two is the second...,<p>Half-Life&reg; 2: Episode Two is the second...,Half-Life&reg; 2: Episode Two is the second in...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,single-player steamachievements captionsavaila...,125.891395
19,game,Team Fortress 2,440.0,0.0,True,"<p><strong>""The most fun you can have online""<...","<p><strong>""The most fun you can have online""<...",Nine distinct classes provide a broad range of...,"

# Testing Individual Request

In [2]:
# This is more of a complete list however, it might show apps that are not 'GAMES'
# https://steamcommunity.com/discussions/forum/1/3017941618731696640/
# When using the storefront api, it is best to be getting your list appids from this interface IStoreService and then the method GetAppList instead.
# https://steamapi.xpaw.me/#IStoreService/GetAppList

# test_url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
# test_df = requests.get(url = test_url).json()
# test_df = pd.DataFrame(test_df)
# test_df = json_normalize(test_df['applist']['apps'])
# test_df = test_df.sort_values(by=['appid'], ascending = True)
# test_df = test_df.reset_index()
# test_df

In [45]:
app_detail_url = 'http://store.steampowered.com/api/appdetails?appids=570'
app_detail = requests.get(url = app_detail_url).json()

app_df = pd.DataFrame(app_detail)
app_df = json_normalize(app_df['570'])
app_df = app_df.drop(1)
app_df

,type,name,steam_appid,required_age,is_free,dlc,detailed_description,about_the_game,short_description,supported_languages,...,platforms.linux,metacritic.score,metacritic.url,recommendations.total,release_date.coming_soon,release_date.date,support_info.url,support_info.email,content_descriptors.ids,content_descriptors.notes
0,game,Dota 2,570.0,0.0,True,"[1241930, 652720]",<strong>The most-played game on Steam.</strong...,<strong>The most-played game on Steam.</strong...,"Every day, millions of players worldwide enter...","Bulgarian, Czech, Danish, Dutch, English<stron...",...,True,90.0,https://www.metacritic.com/game/pc/dota-2?ftag...,14327.0,False,"9 Jul, 2013",,,[],NaN


In [46]:
app_df.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free', 'dlc',
       'detailed_description', 'about_the_game', 'short_description',
       'supported_languages', 'reviews', 'header_image', 'capsule_image',
       'capsule_imagev5', 'website', 'developers', 'publishers', 'packages',
       'package_groups', 'categories', 'genres', 'screenshots', 'movies',
       'background', 'background_raw', 'pc_requirements.minimum',
       'mac_requirements.minimum', 'linux_requirements.minimum',
       'platforms.windows', 'platforms.mac', 'platforms.linux',
       'metacritic.score', 'metacritic.url', 'recommendations.total',
       'release_date.coming_soon', 'release_date.date', 'support_info.url',
       'support_info.email', 'content_descriptors.ids',
       'content_descriptors.notes'],
      dtype='object')

In [47]:
app_df['metacritic.score']

0    90.0
Name: metacritic.score, dtype: float64

In [48]:
app_df['recommendations.total']

0    14327.0
Name: recommendations.total, dtype: float64

In [49]:
app_review_url = 'https://store.steampowered.com/appreviews/10?json=1&language=all&purchase_type=all&num_per_page=0'
app_review = requests.get(url = app_review_url).json()['query_summary']
review_df = pd.json_normalize(app_review)

In [50]:
review_df

,num_reviews,review_score,review_score_desc,total_positive,total_negative,total_reviews
0,0,9,Overwhelmingly Positive,218456,5622,224078


In [51]:
review_df['review_score']

0    9
Name: review_score, dtype: int64

In [52]:
app_review

{'num_reviews': 0,
 'review_score': 9,
 'review_score_desc': 'Overwhelmingly Positive',
 'total_positive': 218456,
 'total_negative': 5622,
 'total_reviews': 224078}